# Public API V2 <a class="anchor" id="notebook-head"></a>

## Введение <a class="anchor" id="introduction"></a>

Этот ноутбук, посвящен
[Public API V2](https://docs.sbercloud.ru/aicloud/mlspace/concepts/api.html) 
 платформы 
[ML Space](https://sbercloud.ru/ru/aicloud/mlspace).

ML Space предоставляет свой 
[REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) 
для того, чтобы пользователи могли частично или полностью автоматизировать свои пайплайны. Использование API позволяет оптимизировать процесс разработки, избавившись от рутинных повторяющихся манипуляций с данными и моделями через 
[ML Space UI](https://docs.sbercloud.ru/aicloud/mlspace/index.html).

Например: у пользователя есть уже заготовленный скрипт. При незначительных модификациях он сможет взаимодействовать с ML Space посредством REST API, что сэкономит время.

Помимо [документации по REST API](https://api.aicloud.sbercloud.ru/public/v2/redoc), мы предлагаем несколько описанных примеров использования.

В этом ноутбуке мы рассмотрим ряд операций, которые вы можете выполнять при помощи REST API, а именно:
- Авторизация пользователя 
- Работа с данными
    - Выгрузка содержания бакета
    - Копирование данных из S3 на NFS
    - Копирование данных из NFS на S3
- Базовые Docker-образы
    - Получение списка базовых Docker-образов
- Задачи на кластере
    - Получение списка задач
    - Запуск задачи
    - Выгрузка логов задачи
- Inference-методы
    - Получение списка сервисов
    - Получение информации по сервису
    - Создание образа для инференс-сервиса
    - Просмотр статуса сборки образа для инференс-сервиса
    - Просмотр логов сборки образа для инференс-сервиса
    - Создание деплоя на основе собранного образа
    - Отправка запроса к созданному сервису

До рассмотрения примеров необходимо проверить, соблюдены ли некоторые предварительные требования, а именно:

* <span style="color:green;font-weight:bold">(Рекомендуется)</span> Ноутбук предпочтительно запускать в Jupyter Lab. Jupyter Lab, в отличие от Jupyter Notebook, умеет качественно визуализировать JSON-структуры, которые часто встречаются в этом ноутбуке.

* <span style="color:green;font-weight:bold">(Рекомендуется)</span> Лучше, чтобы рядом с самим ноутбуком находился JSON-файл с необходимыми авторизационными данными от вашего аккаунта. Это сделает использование удобнее. Вы можете не создавать такой файл. Тогда необходимо иметь при себе логин и пароль от вашего аккаунта, чтобы авторизоваться.

* У вас должен быть ```X-Api-Key```, он же ```GWAPI_KEY```. Более подробно — [в пользовательской документации ML Space](https://docs.sbercloud.ru/aicloud/mlspace/concepts/api.html#id4).

* У вас должен быть ```X-Workspace-Id```. Используется практически во всех REST-запросах в Public API V2.

В начале ноутбука есть содержание, в каждом последующем разделе — обратная ссылка на содержание.

Приятной работы!

# Содержание <a class="anchor" id="toc"></a>

* [Public API V2](#notebook-head)
    * [Введение](#introduction)
* [Все необходимые библиотеки](#all-imports-necessary)
* [Вспомогательные функции и переменные](#auxiliary-stuff)
* [Предварительные требования](#prerequisites)
    * [JSON-файл с необходимыми авторизационными данными](#prerequisites-part-1)
    * [X-Api-Key](#prerequisites-part-2)
    * [X-Workspace-Id](#prerequisites-part-3)
* [Авторизация](#authorization)
* [Работа с данными](#s3)
    * [Выгрузить содержание бакета](#s3-get-bucket-content-structure)
    * [Копировать данные из S3 на NFS](#s3-copy-to-nfs)
    * [Копировать данные из NFS на S3](#s3-copy-from-s3)
* [Базовые Docker-образы](#docker-images)
    * [Получить список базовых Docker-образов](#docker-images-get)
* [Задачи на кластере](#jobs)
    * [Получить список задач](#jobs-get)
    * [Запустить задачу](#jobs-run)
    * [Выгрузить логи задачи](#jobs-job-logs)
* [Inference-методы](#inference)
    * [Получить список сервисов](#inference-get-services-list)
    * [Получить информацию по сервису](#inference-get-service-info)
    * [Создать образ для инференс-сервиса](#inference-create-an-image)
    * [Просмотреть статус сборки образа для инференс-сервиса](#inference-image-creation-status)
    * [Просмотреть  логи сборки образа для инференс-сервиса](#inference-image-creation-logs)
    * [Создать деплой на основе собранного образа](#inference-create-service)
    * [Отправить запрос к созданному сервису](#inference-get-a-response-from-service)

# Все необходимые библиотеки ([содержание](#toc))<a class="anchor" id="all-imports-necessary"></a>

Стандарт отрасли — импорты библиотек в одной ячейке в самом начале ноутбука, чтобы код был более читаемым и чистым.

Так мы и поступим.

In [ ]:
import os

import json

import copy

import boto3

import requests

import client_lib

from IPython.display import JSON

# Вспомогательные функции и переменные ([содержание](#toc))<a class="anchor" id="auxiliary-stuff"></a>

In [ ]:
def wipe_out_sensitive_information(func):
    
    def recursive_information_deletion(credentials_dict):
        sensitive_info_fields = [
            "id", "gwapi-key",
            "client_id", "client_secret",
            "access_key_id", "secret_access_key",
            "endpoint_url", "bucket_name",
            "access_token", "refresh_token",
            "s3_endpoint", "aws_access_key_id",
            "aws_secret_access_key"
        ]
        
        if isinstance(credentials_dict, dict):
            for k, v in credentials_dict.items():
                if isinstance(v, dict):
                    new_v = recursive_information_deletion(v)
                    
                    credentials_dict[k] = new_v
                elif isinstance(v, str):
                    if k in sensitive_info_fields:
                        credentials_dict[k] = "[SENSITIVE INFORMATION, WIPED OUT]"
                elif isinstance(v, list):
                    for i, item in enumerate(v):
                        new_v_i = recursive_information_deletion(item)
                        
                        v[i] = new_v_i
                        
        return credentials_dict

    def wrapper(credentials_dict):
        credentials_dict_copy = copy.deepcopy(credentials_dict)
        
        credentials_dict_copy["[THIS IS A COPY]"] = True
        
        credentials_dict_copy = recursive_information_deletion(credentials_dict_copy)
        
        return_value = func(credentials_dict_copy)
        
        return return_value
    
    return wrapper

In [ ]:
JSON = wipe_out_sensitive_information(JSON)

In [ ]:
host = "https://api.aicloud.sbercloud.ru"

rest_call_template = "{}/public/v2/{}"

# Предварительные требования ([содержание](#toc))<a class="anchor" id="prerequisites"></a>

## JSON-файл с необходимыми авторизационными данными ([содержание](#toc))<a class="anchor" id="prerequisites-part-1"></a>

In [ ]:
with open("credentials_dict_example.json", "r") as f:
    credentials_dict = json.load(f)

In [ ]:
JSON(credentials_dict)

In [ ]:
with open("credentials_dict.json", "r") as f:
    credentials_dict = json.load(f)

In [ ]:
credentials = {
    "client_id": credentials_dict["workspaces"][0]["client_id"],
    "client_secret": credentials_dict["workspaces"][0]["client_secret"]
}

## X-Api-Key ([содержание](#toc))<a class="anchor" id="prerequisites-part-2"></a>

```X-Api-Key``` (он же ```GWAPI_KEY```) — это клиентский ключ доступа к API. Подробнее — [в пользовательской документации](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__develop-func.html)). Он индивидуален для каждого [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html) в рамках аккаунта пользователя.

При создании нового Jupyter Server в окружении создается переменная ```GWAPI_KEY```, куда и записывается значение ```X-Api-Key```.

Есть несколько способов получить эту сущность.

1. Через Python

    ```python
    import os
    print(os.environ["GWAPI_KEY"])
    ```

2. Через командную оболочку (bash, sh, zsh и др.):

    ```bash
    echo $GWAPI_KEY
    ```

3. Через bash-magic команду в Jupyter Notebook/Lab:

    ```bash
    %%bash
    echo $GWAPI_KEY
    ```

    или

    ```bash
    ! echo $GWAPI_KEY
    ```

Далее мы воспользуемся **первым** способом.

## X-Workspace-Id ([содержание](#toc))<a class="anchor" id="prerequisites-part-3"></a>

```X-Workspace-Id``` — это уникальный идентификатор [воркспейса](https://docs.sbercloud.ru/aicloud/mlspace/concepts/profile__workspace.html), в котором вы работаете.

Подробнее об этом поле — в соответствующем разделе [пользовательской документации](https://docs.sbercloud.ru/aicloud/mlspace/concepts/deployments__send-http-requests-to-service.html?highlight=workspace%20id#http).

# Авторизация ([содержание](#toc))<a class="anchor" id="authorization"></a>

In [ ]:
endpoint = "service_auth"

json_body = json.dumps(credentials)

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    data=json_body
)

auth_response_json = json.loads(response.text)

In [ ]:
JSON(auth_response_json)

# Работа с данными ([содержание](#toc))<a class="anchor" id="s3"></a>

## Выгрузить содержание бакета ([содержание](#toc))<a class="anchor" id="s3-get-bucket-content-structure"></a>

In [ ]:
s3 = boto3.client(
    service_name="s3",
    aws_access_key_id=credentials_dict["workspaces"][0]["access_key_id"],
    aws_secret_access_key=credentials_dict["workspaces"][0]["secret_access_key"],
    endpoint_url=credentials_dict["workspaces"][0]["endpoint_url"]
)

In [ ]:
resp = s3.list_objects(Bucket=credentials_dict["workspaces"][0]["bucket_name"])

In [ ]:
if "Contents" in resp:
    bucket_content = {}

    for item in resp["Contents"]:
        sub_folders = item["Key"].split("/")

        temp = bucket_content

        for sub_folder in sub_folders:
            if sub_folder not in temp:
                temp[sub_folder] = {}

                temp = temp[sub_folder]
            else:
                temp = temp[sub_folder]

In [ ]:
JSON(bucket_content)

## Копировать данные из S3 на NFS ([содержание](#toc))<a class="anchor" id="s3-copy-to-nfs"></a>

In [ ]:
s3.download_file(
    credentials_dict["workspaces"][0]["bucket_name"],
    "test_script.py",
    "/home/jovyan/test_script.py"
)

In [ ]:
! ls -alh /home/jovyan/ | grep test_script.py

## Копировать данные из NFS на S3 ([содержание](#toc))<a class="anchor" id="s3-copy-from-s3"></a>

In [ ]:
s3.upload_file(
    "/home/jovyan/env_dependencies_tree",
    credentials_dict["workspaces"][0]["bucket_name"],
    "env_dependencies_tree",
)

# Базовые Docker-образы ([содержание](#toc))<a class="anchor" id="docker-images"></a>

## Получить список базовых Docker-образов ([содержание](#toc))<a class="anchor" id="docker-images-get"></a>

In [ ]:
endpoint = "service/base_mt_images"

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url
)

response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

# Задачи на кластере ([содержание](#toc))<a class="anchor" id="jobs"></a>

## Получить список задач ([содержание](#toc))<a class="anchor" id="jobs-get"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

jobs_response_json = json.loads(response.text)

In [ ]:
JSON(jobs_response_json)

## Запустить задачу ([содержание](#toc))<a class="anchor" id="jobs-run"></a>

In [ ]:
endpoint = "jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

json_body = {
    "base_image": "registry.aicloud.sbcp.ru/horovod-tf15",
    "script": "/home/jovyan/test_script.py",
    "n_workers": 2,
    "instance_type": a100.1gpu.80vG.12C.96G,
    "flags" : {
        "batch_size": "512",
        "model":"resnet50",
        "xla":"False"
    }
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

job_response_json = json.loads(response.text)

In [ ]:
JSON(job_response_json)

## Выгрузить логи задачи ([содержание](#toc))<a class="anchor" id="jobs-job-logs"></a>

In [ ]:
endpoint = "jobs/{}/logs".format(job_response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

# Inference-методы ([содержание](#toc))<a class="anchor" id="inference"></a>

## Получить список сервисов ([содержание](#toc))<a class="anchor" id="inference-get-services-list"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "services": response_json
}

In [ ]:
JSON(response_json)

## Получить информацию по сервису ([содержание](#toc))<a class="anchor" id="inference-get-service-info"></a>

In [ ]:
endpoint = "inference/v1/{}".format(response_json["services"][2]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
response_json = {
    "service": response_json
}

In [ ]:
JSON(response_json)

## Создать образ для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-create-an-image"></a>

In [ ]:
s3.upload_file(
    "/home/jovyan/requirements.txt",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/requirements.txt",
)

In [ ]:
s3.upload_file(
    "/home/jovyan/dummy_mirror_serving_script.py",
    credentials_dict["workspaces"][0]["bucket_name"],
    "inference_folder/dummy_mirror_serving_script.py",
)

In [ ]:
endpoint = "inference/build/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "base_image": "registry.aicloud.sbcp.ru/base/horovod-cuda10.0-tf1.15.0",
    "run_script": "dummy_mirror_serving_script.py",
    "requirements_path": "requirements.txt",
    "artifacts_directory": "{}/inference_folder/".format(credentials_dict["workspaces"][0]["bucket_name"]),
    "AWS_ACCESS_KEY_ID": credentials_dict["workspaces"][0]["access_key_id"],
    "AWS_SECRET_ACCESS_KEY": credentials_dict["workspaces"][0]["secret_access_key"],
    "S3_ENDPOINT": credentials_dict["workspaces"][0]["endpoint_url"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
JSON(response_json)

## Просмотреть статус сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-status"></a>

In [ ]:
endpoint = "service/jobs"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

In [ ]:
JSON(json.loads(response.text))

## Просмотреть логи сборки образа для инференс-сервиса ([содержание](#toc))<a class="anchor" id="inference-image-creation-logs"></a>

In [ ]:
endpoint = "service/jobs/{}/logs".format(response_json["job_name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "GET",
    url,
    headers=headers
)

print(response.text)

## Создать деплой на основе собранного образа ([содержание](#toc))<a class="anchor" id="inference-create-service"></a>

In [ ]:
endpoint = "inference/v1/"

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "image": response_json["image"],
    "resources": {
        "cpu": 1,
        "memory": 10,
        "gpu": "1"
    },
    "replicas": {
        "min": 1,
        "max": 2
    }
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
response_json = json.loads(response.text)

In [ ]:
print(response_json["metadata"]["name"])

## Отправить запрос к созданному сервису ([содержание](#toc))<a class="anchor" id="inference-get-a-response-from-service"></a>

In [ ]:
endpoint = "inference/v1/predict/{}/{}/".format(response_json["metadata"]["name"], response_json["metadata"]["name"])

headers = {
    "x-api-key": credentials_dict["workspaces"][0]["gwapi-key"],
    "authorization": auth_response_json["token"]["access_token"],
    "x-workspace-id": credentials_dict["workspaces"][0]["id"],
    "Content-Type": "application/json",
    "Accept": "application/json"
}

json_body = {
    "instances": [
        {
            "image_link": "1.jpg"
        },
        {
            "image_link": "2.jpg"
        },
        {
            "image_link": "3.jpg"
        }
    ]
}

url = rest_call_template.format(host, endpoint)
url

In [ ]:
response = requests.request(
    "POST",
    url,
    headers=headers,
    json=json_body
)

In [ ]:
JSON(json.loads(response.text))